In [ ]:
import matplotlib.pyplot as plt




def display_image(ax, image):
    img = mpimg.imread(image_path)
    ax.imshow(img)
    ax.axis('off')  # Turn off axis for image subplot

In [ ]:
%load_ext autoreload
%autoreload 2

from m3util.viz.arrows import DrawArrow
from m3util.viz.layout import layout_subfigures_inches
import matplotlib.pyplot as plt


# Dictionary defining the subfigures with positions in inches
subfigures_dict = {
    "AFM_Image": {
        "position": (0.044, 6.5 - 3.2131, 2.0972, 2.0972),
        "skip_margin": True,
    },
    "real_imag": {"position": (2.2014, 6.5 - 2.1364, 2.0972, 2.0972)},
    "amp_phase": {"position": (4.3588, 6.5 - 2.1364, 2.0972, 2.0972)},
    "BE": {"position": (2.2014, 6.5 - 4.2942, 2.0972, 2.0972)},
    "FFT": {"position": (4.3588, 6.5 - 4.2942, 2.0972, 2.0972)},
    "Bipolar": {"position": (0.0532, 6.5 - 6.4609, 4.2361, 2.0972)},
    "Hysteresis": {"position": (4.3588, 6.5 - 6.4609, 2.0972, 2.0972)},
}



# Create a custom figure of size (10 inches width, 6 inches height)
fig, ax = layout_subfigures_inches(size=(6.5, 6.5), subfigures_dict=subfigures_dict)

arrow = DrawArrow(
    fig,
    start_pos=(1.0926, 6.5 - 1),
    end_pos=(2.2014, 6.5-0.4972),
    text="FFT",
    text_position="center",
    text_alignment="center",
    vertical_text_displacement=45,
)



arrow.draw()



In [ ]:
arrow._get_text_position()